In [1]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import date

In [2]:
url = "https://covid-193.p.rapidapi.com/statistics"

In [3]:
headers = {
    'x-rapidapi-host': "covid-193.p.rapidapi.com",
    'x-rapidapi-key': "3ea1c14b00mshb6cedc9f4de354bp172633jsnc0ef76d6f7bd"
    }

In [4]:
result = requests.get(url, headers=headers)

In [5]:
pretty_json = json.loads(result.text)

In [6]:
all_df = pd.json_normalize(pretty_json['response'])

In [7]:
print(json.dumps(pretty_json, indent=2))

{
  "get": "statistics",
  "parameters": [],
  "errors": [],
  "results": 228,
  "response": [
    {
      "continent": "Asia",
      "country": "China",
      "population": 1439323776,
      "cases": {
        "new": "+20",
        "active": 208,
        "critical": 2,
        "recovered": 80628,
        "1M_pop": "59",
        "total": 85470
      },
      "deaths": {
        "new": null,
        "1M_pop": "3",
        "total": 4634
      },
      "tests": {
        "1M_pop": "111163",
        "total": 160000000
      },
      "day": "2020-10-05",
      "time": "2020-10-05T22:15:06+00:00"
    },
    {
      "continent": "Europe",
      "country": "Italy",
      "population": 60438292,
      "cases": {
        "new": "+2257",
        "active": 58903,
        "critical": 323,
        "recovered": 232681,
        "1M_pop": "5420",
        "total": 327586
      },
      "deaths": {
        "new": "+16",
        "1M_pop": "596",
        "total": 36002
      },
      "tests": {
        "1M

In [8]:
all_df.head()

,continent,country,population,day,time,cases.new,cases.active,cases.critical,cases.recovered,cases.1M_pop,cases.total,deaths.new,deaths.1M_pop,deaths.total,tests.1M_pop,tests.total
0,Asia,China,1.439324e+09,2020-10-05,2020-10-05T22:15:06+00:00,+20,208,2,80628,59,85470,None,3,4634,111163,160000000.0
1,Europe,Italy,6.043829e+07,2020-10-05,2020-10-05T22:15:06+00:00,+2257,58903,323,232681,5420,327586,+16,596,36002,195974,11844346.0
2,Europe,Spain,4.675956e+07,2020-10-05,2020-10-05T22:15:06+00:00,+2099,0,1580,0,18239,852838,+46,689,32225,292770,13689776.0
3,North-America,USA,3.315104e+08,2020-10-05,2020-10-05T22:15:06+00:00,+35398,2571231,14206,4886150,23143,7672310,+318,648,214929,338909,112351771.0
4,Europe,Germany,8.385433e+07,2020-10-05,2020-10-05T22:15:06+00:00,+3086,31341,447,263700,3633,304657,+14,115,9616,202724,16999253.0


In [9]:
all_df.loc[all_df["country"] == "Ukraine"]

,continent,country,population,day,time,cases.new,cases.active,cases.critical,cases.recovered,cases.1M_pop,cases.total,deaths.new,deaths.1M_pop,deaths.total,tests.1M_pop,tests.total
100,Europe,Ukraine,43663726.0,2020-10-05,2020-10-05T22:15:06+00:00,+3774,124554,177,101252,5273,230236,+33,101,4430,54559,2382259.0


In [10]:
all_df.shape

(228, 16)

In [11]:
cases_df = all_df.copy().drop(['tests.1M_pop', 
             'deaths.1M_pop', 
             'deaths.new', 
             'cases.recovered',
             'cases.critical', 
             'cases.active', 
             'cases.1M_pop',
             'cases.new',
             'time'                 
            ],
            axis = 1
           )

In [12]:
cases_df.dtypes

continent        object
country          object
population      float64
day              object
cases.total       int64
deaths.total      int64
tests.total     float64
dtype: object

In [13]:
cases_df.population = cases_df.population.fillna(0).astype(int)

In [14]:
cases_df

,continent,country,population,day,cases.total,deaths.total,tests.total
0,Asia,China,1439323776,2020-10-05,85470,4634,160000000.0
1,Europe,Italy,60438292,2020-10-05,327586,36002,11844346.0
2,Europe,Spain,46759558,2020-10-05,852838,32225,13689776.0
3,North-America,USA,331510437,2020-10-05,7672310,214929,112351771.0
4,Europe,Germany,83854334,2020-10-05,304657,9616,16999253.0
...,...,...,...,...,...,...,...
223,Asia,Yemen,29993034,2020-10-05,2041,592,NaN
224,Asia,Tajikistan,9592085,2020-10-05,9974,78,NaN
225,Africa,Comoros,874268,2020-10-05,487,7,NaN
226,Africa,Lesotho,2146630,2020-10-05,1683,39,19428.0


In [15]:
cases_europe_df = cases_df.loc[cases_df['continent'] == 'Europe'].reset_index().drop(['index', 'continent'], axis = 1)

In [16]:
cases_europe_df.drop(cases_europe_df.loc[cases_europe_df['country']=='Europe'].index, inplace=True)
cases_europe_df = cases_europe_df.sort_values(by=['country']).reset_index(drop = True)
cases_europe_df

,country,population,day,cases.total,deaths.total,tests.total
0,Albania,2876966,2020-10-05,14410,400,88622.0
1,Andorra,77298,2020-10-05,2370,53,137457.0
2,Austria,9019845,2020-10-05,48896,818,1698268.0
3,Belarus,9448502,2020-10-05,80696,862,1949656.0
4,Belgium,11602851,2020-10-05,130235,10064,3402761.0
5,Bosnia-and-Herzegovina,3275378,2020-10-05,28449,888,248975.0
6,Bulgaria,6934471,2020-10-05,21870,854,546789.0
7,Channel-Islands,174280,2020-10-05,686,48,123884.0
8,Croatia,4098517,2020-10-05,17797,300,328392.0
9,Czechia,10714212,2020-10-05,83696,757,1457315.0


In [19]:
if date.today().weekday() == 0:
    cases_YYYY_MM_DD = cases_europe_df.copy().drop(['deaths.total','day'], axis = 1)
    cases_YYYY_MM_DD = cases_YYYY_MM_DD.rename(columns={'cases.total' : f'{date.today()}'})
    path = f"../Data/Cases_eu_week_{date.today()}.csv"
    cases_YYYY_MM_DD.to_csv(path)
else:
    week = date.today() - pd.to_timedelta((date.today().weekday()), unit='d')
    path = f"../Data/Cases_eu_week_{week}.csv"
    cases_YYYY_MM_DD = pd.read_csv(path, index_col=0)
    cases_YYYY_MM_DD[f'{date.today()}'] = cases_europe_df.copy()['cases.total']
    cases_YYYY_MM_DD['tests.total'] = cases_europe_df.copy()['tests.total' ]
    cases_YYYY_MM_DD.to_csv(path)

In [20]:
cases_YYYY_MM_DD

,country,population,2020-10-06,tests.total
0,Albania,2876966,14410,88622.0
1,Andorra,77298,2370,137457.0
2,Austria,9019845,48896,1698268.0
3,Belarus,9448502,80696,1949656.0
4,Belgium,11602851,130235,3402761.0
5,Bosnia-and-Herzegovina,3275378,28449,248975.0
6,Bulgaria,6934471,21870,546789.0
7,Channel-Islands,174280,686,123884.0
8,Croatia,4098517,17797,328392.0
9,Czechia,10714212,83696,1457315.0
